# Building The CNN And Learning Almost Everything About It.

#### 1. Why CNN and Not MLP For Images ?
#### 2. How CNN works ? - > Layers : Convolutional, Pooling and Fully Connected Layers 
#### 3. How Backprop Happens In CNN ?

![img](../../images/cnn.png)

### 1. Why We Use The CNN And Not MLP For Images ?

CNN Handles the Image datasets better than the MLP 
and there are two main reason to it : ***1. Sparse Connectivety and 2. Parameter Shairng***


`Sparse Connectivity :` Sparse connectivity point towards that each neuron in a convolutional layer of a CNN is only connected to a small, localized region of the input image, rather than being fully connected to the entire input. This localized region is known as the receptive field.

Local Receptive Fields: In CNNs, each convolutional kernels operates over a small patch of the input image. This means that neurons in the convolutional layer only focus on small regions of the input, making the network more efficient by reducing the number of parameters and computations.

`Parameter Sharing:` The same convolutional filter is applied to entire input image. This parameter sharing allows CNNs to capture features like edges, textures, and patterns regardless of their position in the image.

#### Why not MLP ?

`High Number of Parameters:` For an image of size 32x32 with 3 color channels, a single neuron in the first hidden layer of an MLP would have ***32 × 32 × 3 = 3072*** connections. With a large number of neurons, this quickly becomes impractical.

`Lack of Locality:` MLPs treat all input pixels equally without considering the spatial structure of the image. This lack of locality makes it harder for MLPs to capture local patterns and structures that are important for image recognition tasks.

In [2]:
import numpy as np
import scipy.signal
def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]  # Rotate the filter
    X_orig = np.array(X)
    
    # Pad the input array
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0] + X_orig.shape[0], p[1]:p[1] + X_orig.shape[1]] = X_orig
    
    res = []
    
    # Iterate over the output array dimensions
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) / s[0]) + 1, s[0]):
        row = []
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) / s[1]) + 1, s[1]):
            X_sub = X_padded[i:i + W_rot.shape[0], j:j + W_rot.shape[1]]
            conv_result = np.sum(X_sub * W_rot)
            row.append(conv_result)
        res.append(row)
    
    return np.array(res)

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]
print('Conv2d Implementation:\n',
conv2d(X, W, p=(1, 1), s=(1, 1))) 
print('SciPy Results:\n',scipy.signal.convolve2d(X, W, mode='same'))

Conv2d Implementation:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
SciPy Results:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]
